In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np

# Function to create and compile the model
def create_wrinkles_model(input_shape=(64, 64, 3)):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    base_model.trainable = False

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Function to classify an image
def classify_image(model, img):
    img = cv2.resize(img, (64, 64))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    result = model.predict(img)
    return result[0][0]

# Function to capture frames from live camera
def live_camera_classification():
    model = create_wrinkles_model()
    model.load_weights('wrinkles_model.h5')

    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()

        if not ret:
            print("Failed to capture frame")
            break

        # Classify the captured frame
        prediction = classify_image(model, frame)

        # Display the result on the frame
        if prediction > 0.5:
            cv2.putText(frame, "Wrinkles", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        else:
            cv2.putText(frame, "No Wrinkles", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Display the frame
        cv2.imshow('Live Camera Classification', frame)

        # Press 'q' to exit the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    live_camera_classification()


2024-01-20 11:55:05.282161: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
